In [ ]:
import pandas as pd
import numpy as np
import math
import csv
import warnings
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
import models
import matplotlib.pyplot as plt
import uuid

## Data Processing Functions

In [ ]:
def dataset_truncate(dataset,batch_size,lookback_window, stride):
    truncated_data = {}
    # print(dataset)
    for key,dataframe in dataset.items():
        data_len = len(dataframe)
        data_len_per_batch = ((batch_size)-1)*stride + lookback_window
        num_batches = math.floor(data_len/data_len_per_batch)
        if num_batches==0:
            warnings.warn("Following dataset do not have enough samples: "+key)
            continue
        ideal_data_len = num_batches*data_len_per_batch
        test_case = data_len - ideal_data_len
        if test_case!=0:
            truncated_data[key] = dataframe.drop(dataframe.tail(test_case).index, axis = 0)
            del test_case
        else:
            truncated_data[key] = dataframe
            del test_case
        del data_len
    return truncated_data

## Dataset Generator Functions

In [ ]:
def dataframe_loader(angles, imu, moment, len_cap, mode='FullKinematics', dynamics=None):
    dataset = pd.DataFrame()
    
    if mode.lower()=='fullkinematics':
        for col in angles.columns:
            if 'knee' in col or 'hip_flexion' in col or 'ankle' in col:
                dataset[col]=angles[col]
        
        for col in imu.columns:
            if 'Shank' in col or 'Thigh' in col or 'Pelvis' in col:
                dataset[col]=imu[col]
    
    else:
        for col in angles.columns:
            if 'ankle' in col:
                dataset[col]=angles[col]
        
        for col in imu.columns:
            if 'Shank' in col:
                dataset[col]=imu[col]
                
    if dynamics is not None:
        for col in dynamics.columns:
            if col[0]=='L':
                dataset[col]=dynamics[col]

    dataset['ankle_moment'] = moment['ankle_angle_l_moment']
    dataset.interpolate(method='spline',order=5,axis=0,limit=50,inplace=True)
    dataset.dropna(axis=0, inplace=True)

    if len(dataset)>len_cap:
        n = len(dataset)-len_cap
        dataset.drop(dataset.tail(n).index, inplace = True)
    
    return dataset




def generate_data_dict(subjects,tasks,batch_size=32,lookback_window=50, mode='FullKinematics', stride=1, get_dynamics=0, len_cap=4000, loading_mode='train'):
    ##Generate filenames to look for based on subject and task list
    file_list = {}
    dataset = {}
    address = 'Dataset/'
    for sub in subjects:
        for i in range(len(tasks)):
            task=tasks[i]
            temp = address + sub + '/' + task + '/' + sub + '_' + task + '_'
            temp_a = temp + 'angle.csv'
            temp_i = temp + 'imu_real.csv'
            temp_m = temp + 'moment_filt.csv'
            if get_dynamics==0:
                file_list[sub + '_' + task] = [temp_a,temp_i,temp_m]
                del temp, temp_a, temp_i, temp_m
            else:
                temp_d = temp + 'insole_sim.csv'
                file_list[sub + '_' + task] = [temp_a,temp_i,temp_m,temp_d]
                del temp, temp_a, temp_i, temp_m, temp_d
        del task
    file_names = list(file_list.keys())
    
    ##Check of the dataset for the given configuration of subject and task exist in the system
    checker = '' 
    for item in file_names:
        checker+=item
    checker=checker+str(batch_size)+str(get_dynamics)+str(lookback_window)+str(stride)+mode 
    if loading_mode.lower()=='train':
        with open('Data/train_info.txt','r+') as info_file:
            line=''
            for line in info_file:
                pass
            info = line.rstrip()
            if info==checker:
                return None
            else:
                info_file.write('\n'+checker)
    else:
        with open('Data/test_info.txt','r+') as info_file:
            line=''
            for line in info_file:
                pass
            info = line.rstrip()
            if info==checker:
                return None
            else:
                info_file.write('\n'+checker)

    ##if the file do not exist then it will generate the dataset dictionary
    for i in range(len(file_list)):
        key = file_names[i]
        angle = pd.read_csv(file_list[key][0])
        imu = pd.read_csv(file_list[key][1])
        moment = pd.read_csv(file_list[key][2])
        
        if get_dynamics==0:
            dataset[key] = dataframe_loader(angle,imu,moment,len_cap,mode=mode)
            del key, angle, imu, moment
        else:
            forces = pd.read_csv(file_list[key][3])
            dataset[key] = dataframe_loader(angle,imu,moment,len_cap,mode=mode,dynamics=forces)
            del key, angle, imu, moment, forces
            
    del file_list, file_names, checker
    return dataset




def stack_data(dataset, batch_size, lookback_window, stride, mode='train'):
    truncated_dataset = dataset_truncate(dataset, batch_size, lookback_window, stride)
    training_dataset = pd.DataFrame()
    for key, dataframe in truncated_dataset.items():
        training_dataset = pd.concat([training_dataset,dataframe])
    del truncated_dataset
    training_dataset=training_dataset.to_numpy()
    if mode=='train':
        scaler = MinMaxScaler()
        training_dataset[:,:-1]=scaler.fit_transform(training_dataset[:,:-1])
        # Save the parameters to a file
        with open('Data/scaling_params.pkl', 'wb') as f:
            pickle.dump(scaler, f)
    else:
        scaler = MinMaxScaler()
        with open('Data/scaling_params.pkl', 'rb') as f:
            scaler=pickle.load(f)
            training_dataset[:,:-1]=scaler.transform(training_dataset[:,:-1])
    return training_dataset



def load_data(subject,task,batch_size=32,lookback_window=50, stride=1,with_dynamics=0, data_len_cap=4000, mode="FullKinematics", loading_mode='train'):

    dataset_dict = generate_data_dict(subject, task, get_dynamics=with_dynamics, batch_size=batch_size, lookback_window=lookback_window, stride=stride,mode=mode, len_cap=data_len_cap, loading_mode=loading_mode)
    if dataset_dict==None:
        print("Loading local stored data")
        if loading_mode.lower()=='train':
            dataset = np.load('Data/training_dataset.npy')
        else:
            dataset = np.load('Data/testing_dataset.npy')
    else:
        print('creating_data')
        dataset=stack_data(dataset_dict, batch_size=batch_size,lookback_window=lookback_window,stride=stride,mode=loading_mode)
        if loading_mode.lower()=='train':
            np.save('Data/training_dataset.npy',dataset)
        else:
            np.save('Data/testing_dataset.npy',dataset)
    feature_count = np.shape(dataset)[1]-1
    print('Sample Points= ', np.shape(dataset)[0])
    print('Feature_Count= ', feature_count)
    del dataset_dict
    return dataset,feature_count

def reshape_IO(inputs,outputs, window_size=100, effective_window=1, skip_sz=5):
    data_length = len(inputs)
    output_length = window_size-effective_window+1
    X = []
    Y = []
    for i in range(window_size,data_length,skip_sz):
        temp = inputs[i-window_size:i,:]
        temp2 = outputs[i-output_length:i]
        X.append(temp)
        Y.append(temp2)
    X= np.array(X)
    Y= np.array(Y)
    del data_length
    return X,Y

#### Plotting Function

In [ ]:
def predictor(pred_output,test_output,name='LSTM',time_window=3,init=1, graph='on'):  #prediction plotter
    # print(pred_output.shape,test_output.shape)
    rmse = mean_squared_error(pred_output,test_output)**0.5
    plot_add = 'Plots/'
    if graph=='on':
        unique_id = uuid.uuid1()
        data_pts = int(200*time_window)
        time = np.linspace(0, time_window, data_pts)
        x=1
        k=init
        plt.plot(time,pred_output[k*x:k*x+data_pts], time,test_output[k*x:k*x+data_pts],'--')
        plt.xlabel("Time (seconds)")
        plt.ylabel("Ankle Torque (N.m/Kg)")
        plt.title("%s | RMSE: %f" %(name,rmse))
        plt.legend(['Predicted','True value'])
        plt.savefig(plot_add+f"plot_{unique_id}.png")
        plt.show()
    return rmse

## Sequence Generator Parameters

In [ ]:
batch_sz_train = 32
lookback_train = 100
stride_train = 3
batch_sz_test = 32
lookback_test = 100
stride_test = 6
num_epochs=25


### Feature Importance

In [ ]:
def features_importance(X_train, y_train):
    # configure to select all features
    fs = SelectKBest(score_func=f_regression, k='all')
    # learn relationship from training data
    fs.fit(X_train, y_train)
    scores = []
    for i in range(len(fs.scores_)):
        scores.append(fs.scores_[i])
    plt.bar([i for i in range(len(fs.scores_))], fs.scores_)
    plt.show()
    return scores

## Task Shortlisting - Feed Forward Method

In [ ]:
def training_task_generator(task_list,train_task_idx,preselected_tasks_idx=[0,1]):
    train_task = []
    holdout_task = task_list.copy()
    if len(preselected_tasks_idx)!=0:
        for item in preselected_tasks_idx:
            train_task.append(task_list[item])
            holdout_task.remove(task_list[item])

    if train_task_idx==None:
        return train_task, holdout_task  
          
    if train_task_idx not in preselected_tasks_idx:
        train_task.append(task_list[train_task_idx])
        holdout_task.remove(task_list[train_task_idx])
        
    return train_task, holdout_task

def test_model_perf(train_subject, train_task, test_subject, holdout_task, with_dynamics=0, graph_switch='off'):
    filepath="Model_weights/best_weights.h5"
    train_dataset,feature_count = load_data(train_subject,train_task,batch_size=batch_sz_train,lookback_window=lookback_train, stride=stride_train,with_dynamics=with_dynamics, data_len_cap=4000, loading_mode='train')
    trainX,trainY=reshape_IO(train_dataset[:,:-1],train_dataset[:,-1],window_size=lookback_train,skip_sz=stride_train)
    test_dataset,feature_count = load_data(test_subject,holdout_task,batch_size=batch_sz_test,lookback_window=lookback_test, stride=stride_test,with_dynamics=with_dynamics, data_len_cap=1000, loading_mode='test')
    testX,testY = reshape_IO(test_dataset[:,:-1], test_dataset[:,-1], window_size=lookback_test,skip_sz=stride_test)
    model=models.gen_lstm_model(feature_count)
    history=model.fit(trainX, trainY, batch_size=batch_sz_train, validation_data=(testX,testY), epochs=num_epochs)
    mse = min(history.history['val_mse'])
    return mse


## load the list of all the available tasks
with open('Data/task_list.csv', 'r') as tasks:
    task_list = tasks.read().splitlines()

#Define the list of subjects for testing and training dataset
train_subject =  ['AB01', 'AB02', 'AB03']
test_subject =  ['AB05']

finalized_tasks_idx = [2,6,7,13,15] ## define the indexes based on previous experiments if no previous experiment initialize empty array
finalized_tasks = [] ## create a list with name of finalized tasks
for item in finalized_tasks_idx:
    finalized_tasks.append(task_list[item])

#### Individual task performance evaluation loop

In [ ]:
req_num_train_task = 5
num_req_iterations = req_num_train_task-len(finalized_tasks_idx)

for i in range(num_req_iterations):
    locally_best_task_idx=0
    key=10
    for j in range(len(task_list)):
        print(i*len(task_list)+j+1, 'of', num_req_iterations*len(task_list))
        if j not in finalized_tasks_idx:
            train_task, holdout_task = training_task_generator(task_list,train_task_idx=j,preselected_tasks_idx=finalized_tasks_idx)
            mse = test_model_perf(train_subject,train_task,test_subject,holdout_task,with_dynamics=1)
            if mse<key:
                key=mse
                locally_best_task_idx = j
            with open('feed_forward_task_selection.csv', 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([mse, task_list[j],finalized_tasks])
    finalized_tasks_idx.append(locally_best_task_idx)
    finalized_tasks.append(task_list[locally_best_task_idx])

print(finalized_tasks)

## Load the dataset

In [ ]:
## load the list of all the available tasks
with open('Data/task_list.csv', 'r') as tasks:
    task_list = tasks.read().splitlines()

#Define the list of subjects for testing and training dataset
train_subject =  ['AB01', 'AB02', 'AB03']
test_subject =  ['AB01', 'AB02', 'AB03']

finalized_tasks_idx = [2,6,7,13,15,23] ## define the indexes based on previous experiments if no previous experiment initialize empty array

train_task, holdout_task = training_task_generator(task_list,train_task_idx=None,preselected_tasks_idx=finalized_tasks_idx)
## Load dataset
train_dataset,feature_count = load_data(train_subject,train_task,batch_size=batch_sz_train,lookback_window=lookback_train, stride=stride_train,with_dynamics=1, data_len_cap=4000, mode="Exomode", loading_mode='train')
trainX,trainY=reshape_IO(train_dataset[:,:-1],train_dataset[:,-1],window_size=lookback_train,effective_window=lookback_train, skip_sz=stride_train)

val_dataset,feature_count = load_data(test_subject,holdout_task,batch_size=batch_sz_test,lookback_window=lookback_test, stride=stride_test,with_dynamics=1, data_len_cap=4000, mode="Exomode", loading_mode='test')
valX,valY = reshape_IO(val_dataset[:,:-1], val_dataset[:,-1], window_size=lookback_test, effective_window=lookback_test, skip_sz=stride_test)

# del train_dataset, test_dataset

print("Training Data Shape",trainX.shape, trainY.shape)
print("Validation Data Shape",valX.shape, valY.shape)

## Train the model and predict

#### LSTM Model

In [ ]:
lstm_filepath = 'Model_weights/best_weights_lstm.h5'
lstm_model = models.gen_lstm_model(features=feature_count, sequence_length=lookback_train)
history_lstm = lstm_model.fit(trainX,trainY, validation_data=(valX,valY),epochs=num_epochs, batch_size=32, callbacks=[models.gen_checkpoint(checkpoint_filepath=lstm_filepath)])
lstm_model.load_weights(lstm_filepath)

#### FCNN Model

In [ ]:
fcnn_filepath = 'Model_weights/best_weights_fcnn.h5'
fcnn_model = models.gen_FCNN_model(feature_count)
# history_fcnn=fcnn_model.fit(train_dataset[:,:-1],train_dataset[:,-1], validation_data=(val_dataset[:,:-1],val_dataset[:,-1]), epochs=num_epochs, callbacks=[models.gen_checkpoint(checkpoint_filepath=fcnn_filepath)], batch_size=32)
fcnn_model.load_weights(fcnn_filepath)

In [ ]:
import shap
explainer = shap.Explainer(lstm_model)
shap_values = explainer(train_dataset[:,:-1])
shap.plots.waterfall(shap_values[0])

#### TCN Model

In [ ]:
tcn_filepath = 'Model_weights/best_weights_tcn.h5'
tcn_model, effective_window = models.gen_tcn_model(features=feature_count, num_filters=64, kernel_size=3, num_conv_layers=5)
trainX_tcn, trainY_tcn = reshape_IO(train_dataset[:,:-1],train_dataset[:,-1],window_size=500,skip_sz=300,effective_window=effective_window)
valX_tcn, valY_tcn = reshape_IO(val_dataset[:,:-1],val_dataset[:,-1],window_size=500,skip_sz=300,effective_window=effective_window)
print(trainX_tcn.shape,trainY_tcn.shape)
history_tcn = tcn_model.fit(trainX_tcn,trainY_tcn, validation_data=(valX_tcn,valY_tcn),epochs=600, callbacks=[models.gen_checkpoint(checkpoint_filepath=tcn_filepath)])
valY_tcn.reshape(-1)
tcn_model.load_weights(tcn_filepath)

## Evaluating model on holdout_tasks

In [ ]:
# Test results on holdout-tasks relevant to exoskeleton
with open('Data/test_task_list.csv', 'r') as tasks:
    test_task_list = tasks.read().splitlines()

## Test results on all holdout-tasks
# test_task_list = holdout_task

In [ ]:
rmse_FCNN = []
rmse_TCN = []
rmse_LSTM = []
holdout_tasks = []
test_subject=['AB05']
for item in test_task_list:
    if item not in train_task:
        print
        test_task=[item]
        print(item)
        
        test_dataset,feature_count = load_data(test_subject,test_task,batch_size=32,lookback_window=100, stride=1,with_dynamics=1, data_len_cap=4000, mode="Exomode", loading_mode='test')
        if test_dataset.shape[0]>=500:
            testX,testY = reshape_IO(test_dataset[:,:-1], test_dataset[:,-1], effective_window=100, window_size=100, skip_sz=1)
            testX_tcn,testY_tcn = reshape_IO(test_dataset[:,:-1], test_dataset[:,-1], effective_window=effective_window, window_size=500, skip_sz=500-effective_window)
            predY_fcnn = fcnn_model.predict(test_dataset[:,:-1]).reshape(-1)
            predY_LSTM = lstm_model.predict(testX).reshape(-1)
            predY_tcn = tcn_model.predict(testX_tcn).reshape(-1)
            testY = testY.reshape(-1)
            testY_tcn = testY_tcn.reshape(-1)
            temp1=predictor(predY_fcnn,test_dataset[:,-1],'fcnn_'+item,graph='on', time_window=2,init=50)
            temp2= predictor(predY_LSTM,testY,'lstm_'+item,graph='on', time_window=2,init=50)
            temp3= predictor(predY_tcn,testY_tcn,'tcn_'+item,graph='on', time_window=2,init=50)
            holdout_tasks.append(item)
            rmse_FCNN.append(temp1)
            rmse_LSTM.append(temp2)
            rmse_TCN.append(temp3)

## Plot evaluation results on Holdout Tasks

In [ ]:
avg_FCNN_rmse = np.average(rmse_FCNN)
avg_LSTM_rmse = np.average(rmse_LSTM)
avg_TCN_rmse = np.average(rmse_TCN)
print('Average RMSE FCNN: ', avg_FCNN_rmse)
print('Average RMSE LSTM: ', avg_LSTM_rmse)
print('Average RMSE TCN: ', avg_TCN_rmse)

In [ ]:
x = np.arange(len(holdout_tasks))

# Set up the figure and axis
fig, ax = plt.subplots()

# Plot the three data series as bar charts, with different colors
ax.bar(x - 0.2, rmse_FCNN, color='red', width=0.2, align='center')
ax.bar(x, rmse_LSTM, color='green', width=0.2, align='center')
ax.bar(x + 0.2, rmse_TCN, color='blue', width=0.2, align='center')

# Set the x-axis labels and title
ax.set_xlabel('Ambulation modes')
ax.set_ylabel('RMSE (N.m/Kg)')
ax.set_title('RMSE of hold-out ambulation modes for test subject')

# Set the x-axis ticks to show the sample points
ax.set_xticks(x)
ax.set_xticklabels(holdout_tasks, rotation='vertical')

# Set the legend for the three data series
ax.legend(['FCNN', 'LSTM', 'TCN'])

# Show the plot
plt.show()